In [1]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [2]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

[<Experiment: artifact_location='/Users/teeboy/repos/kmutt/cpe393-mlflow/mlruns/1', creation_time=1743002305076, experiment_id='1', last_update_time=1743002305076, lifecycle_stage='active', name='mlops_nyc_taxi', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1743002209581, experiment_id='0', last_update_time=1743002209581, lifecycle_stage='active', name='Default', tags={}>]

In [3]:
client.create_experiment(name="my-new-experiment")

'2'

In [4]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 100",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [5]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: a8f6baec408c4c99b5fffff835ec36f5, rmse: 28.2370
run id: 1bf261d38a204bdc89667ec3bcbedcc7, rmse: 33.9278
run id: 59bc64ddbc0147c3b7a332ab595cf04c, rmse: 35.6832
run id: 0015ec60ad8347abb5b1029d19e09f87, rmse: 68.5237


In [6]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [8]:
run_id = "1bf261d38a204bdc89667ec3bcbedcc7"  #insert your run id
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc_taxi_xgb")

Registered model 'nyc_taxi_xgb' already exists. Creating a new version of this model...
Created version '2' of model 'nyc_taxi_xgb'.


<ModelVersion: aliases=[], creation_timestamp=1743002671818, current_stage='None', description=None, last_updated_timestamp=1743002671818, name='nyc_taxi_xgb', run_id='1bf261d38a204bdc89667ec3bcbedcc7', run_link=None, source='/Users/teeboy/repos/kmutt/cpe393-mlflow/mlruns/1/1bf261d38a204bdc89667ec3bcbedcc7/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [9]:
#check model version and stages
model_name = "nyc_taxi_xgb"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Staging
version: 2, stage: None


/var/folders/_7/0vl1v1p517j0b_0lbv4d11mw0000gn/T/ipykernel_44851/729671938.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [10]:
model_version = 2
new_stage = "Staging"
# Use client.transition_model_version_stage() function to model to staging.
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False # so version 1 will remain in Staging
)

/var/folders/_7/0vl1v1p517j0b_0lbv4d11mw0000gn/T/ipykernel_44851/2534192036.py:4: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1743002671818, current_stage='Staging', description=None, last_updated_timestamp=1743002745957, name='nyc_taxi_xgb', run_id='1bf261d38a204bdc89667ec3bcbedcc7', run_link=None, source='/Users/teeboy/repos/kmutt/cpe393-mlflow/mlruns/1/1bf261d38a204bdc89667ec3bcbedcc7/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [11]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1743002671818, current_stage='Staging', description='The model version 2 was transitioned to Staging on 2025-03-26', last_updated_timestamp=1743002778163, name='nyc_taxi_xgb', run_id='1bf261d38a204bdc89667ec3bcbedcc7', run_link=None, source='/Users/teeboy/repos/kmutt/cpe393-mlflow/mlruns/1/1bf261d38a204bdc89667ec3bcbedcc7/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [12]:
model_version = 1
new_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False # 
)

from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

/var/folders/_7/0vl1v1p517j0b_0lbv4d11mw0000gn/T/ipykernel_44851/3713450330.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1743002468139, current_stage='Production', description='The model version 1 was transitioned to Production on 2025-03-26', last_updated_timestamp=1743002798431, name='nyc_taxi_xgb', run_id='1bf261d38a204bdc89667ec3bcbedcc7', run_link='', source='/Users/teeboy/repos/kmutt/cpe393-mlflow/mlruns/1/1bf261d38a204bdc89667ec3bcbedcc7/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=1>

Comparing versions and selecting the new "Production" model
In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
Download the DictVectorizer that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
Preprocess the test set using the DictVectorizer so we can properly feed the regressors.
Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
Based on the results, update the "Production" model version accordingly.


In [45]:
# import packages
import numpy as np
from sklearn.metrics import mean_squared_error
import pandas as pd

In [ ]:
# print(mean_squared_error.__doc__)

Mean squared error regression loss.

    Read more in the :ref:`User Guide <mean_squared_error>`.

    Parameters
    ----------
    y_true : array-like of shape (n_samples,) or (n_samples, n_outputs)
        Ground truth (correct) target values.

    y_pred : array-like of shape (n_samples,) or (n_samples, n_outputs)
        Estimated target values.

    sample_weight : array-like of shape (n_samples,), default=None
        Sample weights.

    multioutput : {'raw_values', 'uniform_average'} or array-like of shape             (n_outputs,), default='uniform_average'
        Defines aggregating of multiple output values.
        Array-like value defines weights used to average errors.

        'raw_values' :
            Returns a full set of errors in case of multioutput input.

        'uniform_average' :
            Errors of all outputs are averaged with uniform weight.

    Returns
    -------
    loss : float or array of floats
        A non-negative floating point value (the best 

In [ ]:
# download March 2022 yellow taxi data as `test` dataset
# we used Feb 2022 for validation (which is sometimes also effectively the 'test' set)
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-03.parquet 

In [46]:
# create our functions

def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts) # not fitting the pre-processor again


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))  # Manually compute RMSE
    return {"rmse": rmse}

In [24]:
df = read_dataframe("yellow_tripdata_2022-03.parquet")

In [25]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.') # downloaded locally

'/Users/teeboy/repos/kmutt/cpe393-mlflow/preprocessor'

In [26]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [27]:
X_test = preprocess(df, dv)

In [28]:
target = "duration"
y_test = df[target].values

In [47]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test) # magic command % to clock the test

CPU times: user 14.2 s, sys: 155 ms, total: 14.3 s
Wall time: 2.45 s


{'rmse': np.float64(6.14556862286958)}

In [ ]:
# import sklearn
# print(sklearn.__version__)


1.6.1


In [ ]:
# import sklearn.metrics
# print(dir(sklearn.metrics))


['ConfusionMatrixDisplay', 'DetCurveDisplay', 'DistanceMetric', 'PrecisionRecallDisplay', 'PredictionErrorDisplay', 'RocCurveDisplay', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_base', '_classification', '_dist_metrics', '_pairwise_distances_reduction', '_pairwise_fast', '_plot', '_ranking', '_regression', '_scorer', 'accuracy_score', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'auc', 'average_precision_score', 'balanced_accuracy_score', 'brier_score_loss', 'calinski_harabasz_score', 'check_scoring', 'class_likelihood_ratios', 'classification_report', 'cluster', 'cohen_kappa_score', 'completeness_score', 'confusion_matrix', 'consensus_score', 'coverage_error', 'd2_absolute_error_score', 'd2_log_loss_score', 'd2_pinball_score', 'd2_tweedie_score', 'davies_bouldin_score', 'dcg_score', 'det_curve', 'euclidean_distances', 'explained_variance_score', 'f1_score', 'fbeta_score', 'fowlkes_mallows_s